<a href="https://colab.research.google.com/github/Ngchihieu2003/timeseries/blob/main/kiemtra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM

# Bước 1: Đọc và hợp nhất dữ liệu
gdp = pd.read_csv("GDPC1.csv")
pce = pd.read_csv("PCECC96.csv")

gdp = gdp.rename(columns={"observation_date": "DATE", "GDPC1": "GDP"})
pce = pce.rename(columns={"observation_date": "DATE", "PCECC96": "PCE"})

df = pd.merge(gdp, pce, on="DATE")
df["DATE"] = pd.to_datetime(df["DATE"])
df = df.set_index("DATE").dropna()

# Bước 2: ADF Test
def adf_test(series, name):
    result = adfuller(series)
    print(f"--- ADF Test for {name} ---")
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    for key, value in result[4].items():
        print(f"Critical Value ({key}): {value}")
    print("=> Chuỗi dừng\n" if result[1] <= 0.05 else "=> Chuỗi không dừng\n")

adf_test(df["GDP"], "GDP")
adf_test(df["PCE"], "PCE")

# Bước 3: Johansen Test
print("--- Kiểm định Johansen ---")
johan_test = coint_johansen(df[["GDP", "PCE"]], det_order=0, k_ar_diff=1)
print("Trace Statistic:", johan_test.lr1)
print("Critical Values:")
print(johan_test.cvt)

# Bước 4: VECM
vecm_model = VECM(df[["GDP", "PCE"]], k_ar_diff=1, coint_rank=1)
vecm_result = vecm_model.fit()
print(vecm_result.summary())


--- ADF Test for GDP ---
ADF Statistic: 3.7738224736803865
p-value: 1.0
Critical Value (1%): -3.451552879535732
Critical Value (5%): -2.8708786756338407
Critical Value (10%): -2.571745666091128
=> Chuỗi không dừng

--- ADF Test for PCE ---
ADF Statistic: 4.37344506208366
p-value: 1.0
Critical Value (1%): -3.451552879535732
Critical Value (5%): -2.8708786756338407
Critical Value (10%): -2.571745666091128
=> Chuỗi không dừng

--- Kiểm định Johansen ---
Trace Statistic: [23.79561594  2.38543206]
Critical Values:
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
Det. terms outside the coint. relation & lagged endog. parameters for equation GDP
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
L1.GDP        -0.1607      0.144     -1.115      0.265      -0.443       0.122
L1.PCE         0.0173      0.199      0.087      0.931      -0.372       0.406
Det. terms outside the coint. r

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
